In [1]:
pip install praatio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.2/79.2 KB 2.4 MB/s eta 0:00:00


In [2]:
pip install pydub

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
pip install librosa

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
from praatio import textgrid
from pydub import AudioSegment
import os
import random
import shutil
import json
import librosa

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Step 1: text train split based on the original set provided

In [7]:
##Define the tier name, input/out for audio and textgrid files

tier_name = "channel1_S1" # Replace this with the actual name of the tier you want to work with

root_dir = '/content/drive/MyDrive/Colab Notebooks/AI6127 NLP/GroupProject/Data/' 

# Input original and output target processed transcripts
input_transcript_folder = root_dir + "transcripts/transcript_original"
 
 ## Input and target for audio and target split audio files 
input_audio_folder = root_dir + 'audio/audio_original/'


target_audio_dir = root_dir + 'audio/audio_processed/'

# Check if the target audio directory exists
if not os.path.exists(target_audio_dir):
    # Create the directory if it doesn't exist
    os.makedirs(target_audio_dir)

'src_transcript_dir = \'/content/drive/MyDrive/Colab Notebooks/AI6127 NLP/GroupProject/Data/transcripts/original\'\ndst_transcript_train_dir = \'/content/drive/MyDrive/Colab Notebooks/AI6127 NLP/GroupProject/Data/transcripts/train\'\ndst_transcript_test_dir = \'/content/drive/MyDrive/Colab Notebooks/AI6127 NLP/GroupProject/Data/transcripts/test\'\n#test_train_split(src_transcript_dir,dst_transcript_dir,num);\n\n##splitting audio for train and test\n# src_audio_dir = \'/content/drive/MyDrive/Colab Notebooks/AI6127 NLP/GroupProject/Data/audio/original\'\n\n# not required\n# dst_audio_train_dir = \'/content/drive/MyDrive/Colab Notebooks/AI6127 NLP/GroupProject/Data/audio/train\'\n# dst_audio_test_dir = \'/content/drive/MyDrive/Colab Notebooks/AI6127 NLP/GroupProject/Data/audio/test\'\n\n\ntextgrid_folder_train = "/content/drive/MyDrive/Colab Notebooks/AI6127 NLP/GroupProject/Data/transcripts_processed/train"\ntextgrid_folder_test = "/content/drive/MyDrive/Colab Notebooks/AI6127 NLP/GroupP

In [9]:

def extract_intervals(textgrid_file, tier_name):
 tg = textgrid.openTextgrid(textgrid_file,False)
 tier = tg.getTier(tier_name)
 intervals = [(entry.start, entry.end, entry.label) for entry in tier.entries if entry.label != "<Z>"]
 return intervals


In [10]:
def split_wav_file(input_file, start_time, end_time, output_file):
# Load the input .wav file
    audio = AudioSegment.from_wav(input_file)
    # Convert start and end time to milliseconds
    start_time_ms = start_time * 1000
    end_time_ms = end_time * 1000
    # Slice the audio
    sliced_audio = audio[start_time_ms:end_time_ms]
    # Export the sliced audio to a new .wav file
    sliced_audio.export(output_file, format="wav")

In [12]:
import os
import random

list_of_training_dicts = []
total = 0
# Loop through all training convo files in the folder, slice them into tiny sentence audio
for instance, filename in enumerate(os.listdir(input_transcript_folder)):
    print("completed: ", instance)
    # Check if the current item is a file
    tg_file = os.path.join(input_transcript_folder, filename)
    audio_file = os.path.join(input_audio_folder, filename[:-9]+'.wav')
    tg_content = extract_intervals(tg_file, tier_name)
    for i, e in enumerate(tg_content):
        total += 1
        output_audio_file = target_audio_dir + filename[:-13]+ '-' + str(i) +  '.wav'
        split_wav_file(audio_file, e[0], e[1], output_audio_file)
        d = {'audio_filepath':output_audio_file, 'duration':e[1]-e[0], 'text':e[2] }
        list_of_training_dicts.append(d)
print(f"we have total {total} training instances. successfully processed")


# Loop through all sliced dict data, write to train/validation/test manifest 
train_manifest, validation_manifest, test_manifest = root_dir + 'manifest_train.json', root_dir + 'manifest_validation.json', root_dir + 'manifest_test.json'
with open(train_manifest, 'w') as tm:
    with open(validation_manifest, 'w') as vm:
        with open(test_manifest, 'w') as tem:
            for i, d in enumerate(list_of_training_dicts):
                d['duration'] = round(d['duration'], 2)
                rv = random.random()
                # Test
                if rv  < 0.15: 
                    #json.dump(d, tem)
                    tem.write(str(d)) 
                    tem.write('\n') 
                #validation
                elif rv >= 0.15 and rv < 0.3: 
                    vm.write(str(d))
                    vm.write('\n') 
                # train
                else: 
                    tm.write(str(d))
                    tm.write('\n') 
                if i % 1000 == 0:
                    print("Completed: ", i)




completed:  0
completed:  1
completed:  2
completed:  3
completed:  4
completed:  5
completed:  6
completed:  7
completed:  8
completed:  9
completed:  10
completed:  11
completed:  12
completed:  13
completed:  14
completed:  15
completed:  16
completed:  17
completed:  18
completed:  19
completed:  20
completed:  21
completed:  22
completed:  23
completed:  24
completed:  25
completed:  26
completed:  27
completed:  28
completed:  29
completed:  30
completed:  31
completed:  32
completed:  33
completed:  34
completed:  35
completed:  36
completed:  37
completed:  38
completed:  39
completed:  40
completed:  41
completed:  42
completed:  43
completed:  44
completed:  45
completed:  46
completed:  47
completed:  48
completed:  49
completed:  50
completed:  51
completed:  52
completed:  53
completed:  54
completed:  55
completed:  56
completed:  57


In [20]:
# Print total instance processed
len(list_of_training_dicts)


7557